In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Part 1

In [2]:
def find_max_area(points):
    max_area=0
    for i,pt1 in enumerate(points[:-1]):
        for pt2 in points[i+1:]:
            x1,y1 = pt1
            x2,y2 = pt2
            area = ( abs(x1-x2)+1)*( abs(y1-y2)+1)
            max_area = max(max_area,area)
    return max_area

In [3]:
def part_one(file_path='input_data/test_09.txt'):
    points = []
    with open(file_path,'r') as f:
        for line in f.readlines():
            points.append( [ int(s) for s in line.strip().split(',')  ] )

    area = find_max_area(points)

    return area
           

In [4]:
%%time
part_one()

CPU times: user 1.1 ms, sys: 2.06 ms, total: 3.15 ms
Wall time: 1.68 ms


50

In [5]:
%%time
part_one('input_data/day_09.txt')
        

CPU times: user 37.6 ms, sys: 1.83 ms, total: 39.4 ms
Wall time: 39.5 ms


4771532800

# Part 2


In [1]:
from collections import defaultdict, deque
import numpy as np

class Interior:

    def __init__(self, points):
        self.points = points
        
        edge_points, min_corner, max_corner = self.find_edges()
        
        self.edge_points = edge_points
        self.min_corner = [ p-1 for p in min_corner]
        self.max_corner = [ p+1 for p in max_corner]
        self.presummed_grid = None
        self.point_grid = None
        
        
    def find_edges(self):

        points = self.points
        edge_points = {'x': defaultdict(list),
                       'y': defaultdict(list),
                      }
    
        min_corner= [float('inf'), float('inf')]
        max_corner= [0,0]
        
        for i,pt1 in enumerate(points):
            pt2= points[(i+1)%len(points)]
    
            min_corner = [ min(min_corner[x], pt1[x]) for x in range(2)]
            max_corner = [ max(max_corner[x], pt1[x]) for x in range(2)]
    
            if pt1[0] == pt2[0]:
                start  = min(pt1[1],pt2[1])
                end  = max(pt1[1],pt2[1])
    
                edge_points['x'][pt1[0]].append( (start,end) ) 
    
            elif pt1[1] == pt2[1]:
                start  = min(pt1[0],pt2[0])
                end  = max(pt1[0],pt2[0]) 
                edge_points['y'][pt1[1]].append( (start,end) ) 
        
        return edge_points, min_corner, max_corner

    def check_if_in_edge(self, pt):
        x,y = pt

        for check in self.edge_points['x'][x]:
            if check[0] <= y and check[1] >= y:
                return True
                
        for check in self.edge_points['y'][y]:
            if check[0] <= x and check[1] >= x:
                return True
        return False 
    
    
    def get_neighbors(self, pt):
        lower = self.min_corner
        upper = self.max_corner
        
        neighs = []
        for i, j in [ (0,1),(0,-1),(1,0),(-1,0)]:
            new = [ pt[0]+i ,  pt[1]+j]
            if new[0] >= lower[0] and new[0] <= upper[0]:
                if new[1] >= lower[1] and new[1] <= upper[1]:
                    neighs.append(new)
        return neighs
    
    
    def flood_fill_outer(self,) :
        lower = self.min_corner
        upper = self.max_corner
        
        seen = set([])
        frontier =deque([lower])


        grid={}
        for x in range(lower[0], upper[0]+1):
            grid[x]= defaultdict(int)
            for y in range(lower[1], upper[1]+1):
                grid[x][y]= 0
            
        
        while len(frontier) >0:
            cur = frontier.popleft()            
            grid[cur[0]][cur[1]]= 1
    
            for n in self.get_neighbors(cur):
                x,y = n
                if not self.check_if_in_edge(n) and tuple(n) not in seen:
                    seen.add(tuple(n))
                    frontier.append(n)
                    
        self.point_grid = grid


    def get_presum(self):
        lower = self.min_corner
        upper = self.max_corner
        
        grid={}
        for x in range(lower[0], upper[0]+1):
            grid[x]= defaultdict(int)
            for y in range(lower[1], upper[1]+1):
                p1 = grid[x-1][y] if x-1  in grid.keys() else 0
                p2 = grid[x-1][y-1] if x-1  in grid.keys() else 0
                
                grid[x][y]=  self.point_grid[x][y]+grid[x][y-1] + p1 - p2

        self.presummed_grid = grid
        
        
        
    def check_box_inside(self, pt1, pt2):
        
        min_corner = tuple([ min(pt2[x], pt1[x]) for x in range(2)])
        max_corner = tuple([ max(pt2[x], pt1[x]) for x in range(2)])


        large = self.presummed_grid[max_corner[0]][max_corner[1]]
        small = self.presummed_grid[min_corner[0]-1][min_corner[1]-1]
        
        left =  self.presummed_grid[min_corner[0]-1][max_corner[1]]
        upper =  self.presummed_grid[max_corner[0]][min_corner[1]-1]
        
        total = large + small - left - upper
        
        return total == 0 

    
    def find_max_box(self):
        
        self.flood_fill_outer()
        self.get_presum()
    
        max_area = 0
        
        for i, pt1 in enumerate(self.points):
            for pt2 in self.points[i+1:]:
                if self.check_box_inside(pt1,pt2):
                    x1,y1 = pt1
                    x2,y2 = pt2
                    area = ( abs(x1-x2)+1)*( abs(y1-y2)+1)
                    max_area = max(max_area,area)
                    
        return max_area        
    
        
def part_two(file_path='input_data/test_09.txt'):
    points = []
    with open(file_path,'r') as f:
        for line in f.readlines():
            points.append( [ int(s) for s in line.strip().split(',')  ] )

    check = Interior(points)
    return check.find_max_box()

           

In [2]:
%%time
part_two()

CPU times: user 1.13 ms, sys: 1.41 ms, total: 2.53 ms
Wall time: 1.73 ms


KeyError: 11

In [ ]:
%%time
part_two('input_data/day_09.txt')

In [1]:
from collections import defaultdict, deque
import numpy as np

class Interior:

    def __init__(self, points):
        self.points = points
        self.create_compressed_grid()
        
        edge_points, min_corner, max_corner = self.find_edges()
        
        self.edge_points = edge_points
        self.min_corner = [ p-1 for p in min_corner]
        self.max_corner = [ p+1 for p in max_corner]
        self.presummed_grid = None
        self.point_grid = None

    def create_compressed_grid(self):
        x_set = set([])
        y_set = set([])
        for pt in self.points:
            x,y = pt
            x_set.update([x-1, x, x+1])
            y_set.update([y-1, y, y+1])

        x_coords = list(x_set)
        y_coords = list(y_set)

        x_coords.sort()
        y_coords.sort()

        self.convert_c_to_r = { 'x': x_coords,
                                'y': y_coords}

        
        self.convert_r_to_c = { 'x': { r:i for i,r in enumerate(x_coords)},
                                'y': { r:i for i,r in enumerate(y_coords)}
                              }
        

        compressed = []
        for pt in self.points:
            x,y = pt
            newx = self.convert_r_to_c['x'][x]
            newy = self.convert_r_to_c['y'][y]
            compressed.append((newx,newy))
        self.compressed_pts = compressed
        
        
    def find_edges(self):

        points = self.compressed_pts

        
        edge_points = {'x': defaultdict(list),
                       'y': defaultdict(list),
                      }
        
        
        min_corner= [float('inf'), float('inf')]
        max_corner= [0,0]
        
        for i,pt1 in enumerate(points):
            pt2= points[(i+1)%len(points)]
    
            min_corner = [ min(min_corner[x], pt1[x]) for x in range(2)]
            max_corner = [ max(max_corner[x], pt1[x]) for x in range(2)]
    
            if pt1[0] == pt2[0]:
                start  = min(pt1[1],pt2[1])
                end  = max(pt1[1],pt2[1])
    
                edge_points['x'][pt1[0]].append( (start,end) ) 
    
            elif pt1[1] == pt2[1]:
                start  = min(pt1[0],pt2[0])
                end  = max(pt1[0],pt2[0]) 
                edge_points['y'][pt1[1]].append( (start,end) ) 
        
        return edge_points, min_corner, max_corner

    def check_if_in_edge(self, pt):
        x,y = pt

        for check in self.edge_points['x'][x]:
            if check[0] <= y and check[1] >= y:
                return True
                
        for check in self.edge_points['y'][y]:
            if check[0] <= x and check[1] >= x:
                return True
        return False 
    
    
    def get_neighbors(self, pt):
        lower = self.min_corner
        upper = self.max_corner
        
        neighs = []
        for i, j in [ (0,1),(0,-1),(1,0),(-1,0)]:
            new = [ pt[0]+i ,  pt[1]+j]
            if new[0] >= lower[0] and new[0] <= upper[0]:
                if new[1] >= lower[1] and new[1] <= upper[1]:
                    neighs.append(new)
        return neighs
    
    
    def flood_fill_outer(self,) :
        lower = self.min_corner
        upper = self.max_corner
        
        seen = set([tuple(lower)])
        frontier =deque([lower])


        grid={}
        for x in range(lower[0], upper[0]+1):
            grid[x]= defaultdict(int)
            for y in range(lower[1], upper[1]+1):
                grid[x][y]= 0
            
        
        while len(frontier) >0:
            cur = frontier.popleft()            
            grid[cur[0]][cur[1]]= 1
    
            for n in self.get_neighbors(cur):
                x,y = n
                if not self.check_if_in_edge(n) and tuple(n) not in seen:
                    seen.add(tuple(n))
                    frontier.append(n)
                    
        self.point_grid = grid


    def get_presum(self):
        lower = self.min_corner
        upper = self.max_corner
        
        grid={}
        for x in range(lower[0], upper[0]+1):
            grid[x]= defaultdict(int)
            for y in range(lower[1], upper[1]+1):
                p1 = grid[x-1][y] if x-1  in grid.keys() else 0
                p2 = grid[x-1][y-1] if x-1  in grid.keys() else 0
                
                grid[x][y]=  self.point_grid[x][y]+grid[x][y-1] + p1 - p2

        self.presummed_grid = grid
        
        
        
    def check_box_inside(self, pt1, pt2):
        
        min_corner = tuple([ min(pt2[x], pt1[x]) for x in range(2)])
        max_corner = tuple([ max(pt2[x], pt1[x]) for x in range(2)])


        large = self.presummed_grid[max_corner[0]][max_corner[1]]
        small = self.presummed_grid[min_corner[0]-1][min_corner[1]-1]
        
        left =  self.presummed_grid[min_corner[0]-1][max_corner[1]]
        upper =  self.presummed_grid[max_corner[0]][min_corner[1]-1]
        
        total = large + small - left - upper
        
        return total == 0 

    
    def find_max_box(self):
        
        self.flood_fill_outer()
        self.get_presum()
    
        max_area = 0
        
        for i, pt1 in enumerate(self.compressed_pts):
            for pt2 in self.points[i+1:]:
                if self.check_box_inside(pt1,pt2):
                    x1,y1 = pt1
                    x2,y2 = pt2

                    x1 = self.convert_c_to_r['x'][x1]
                    x2 = self.convert_c_to_r['x'][x2]
                    y1 = self.convert_c_to_r['y'][y1]
                    y2 = self.convert_c_to_r['y'][y2]
                    
                    area = ( abs(x1-x2)+1)*( abs(y1-y2)+1)
                    max_area = max(max_area,area)
                    
        return max_area        
    
        
def part_two(file_path='input_data/test_09.txt'):
    points = []
    with open(file_path,'r') as f:
        for line in f.readlines():
            points.append( [ int(s) for s in line.strip().split(',')  ] )

    check = Interior(points)
    return check.find_max_box()

           